In [ ]:
pip install scikit-lego

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.cluster import AgglomerativeClustering
from sklearn.metrics import silhouette_score, silhouette_samples, rand_score, adjusted_rand_score

In [ ]:
df = pd.read_csv("hf://datasets/maharshipandya/spotify-tracks-dataset/dataset.csv")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
#Checking loading of data frame
df.head(5)

,Unnamed: 0,track_id,artists,album_name,track_name,popularity,duration_ms,explicit,danceability,energy,...,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,track_genre
0,0,5SuOikwiRyPMVoIQDJUgSV,Gen Hoshino,Comedy,Comedy,73,230666,False,0.676,0.4610,...,-6.746,0,0.1430,0.0322,0.000001,0.3580,0.715,87.917,4,acoustic
1,1,4qPNDBW1i3p13qLCt0Ki3A,Ben Woodward,Ghost (Acoustic),Ghost - Acoustic,55,149610,False,0.420,0.1660,...,-17.235,1,0.0763,0.9240,0.000006,0.1010,0.267,77.489,4,acoustic
2,2,1iJBSr7s7jYXzM8EGcbK5b,Ingrid Michaelson;ZAYN,To Begin Again,To Begin Again,57,210826,False,0.438,0.3590,...,-9.734,1,0.0557,0.2100,0.000000,0.1170,0.120,76.332,4,acoustic
3,3,6lfxq3CG4xtTiEg7opyCyx,Kina Grannis,Crazy Rich Asians (Original Motion Picture Sou...,Can't Help Falling In Love,71,201933,False,0.266,0.0596,...,-18.515,1,0.0363,0.9050,0.000071,0.1320,0.143,181.740,3,acoustic
4,4,5vjLSffimiIP26QG5WcN2K,Chord Overstreet,Hold On,Hold On,82,198853,False,0.618,0.4430,...,-9.681,1,0.0526,0.4690,0.000000,0.0829,0.167,119.949,4,acoustic


## Apply cleaning function

Going to look at clustering popular music genres (Pop, Rock,Hip-Hop,Electronic,Indie, Country, Jazz, Latin, Reggae, Blues)


1.   pop
2.   rock
3. hip-hop
4. electronic
5. indie
6. blues
7. county
8. jazz
9. latin
10. reggae



In [ ]:
# Data Cleaning Function (modify)
def clean_data(df):
  clean_df = df.copy()
  clean_df = clean_df.drop('Unnamed: 0', axis=1, errors='ignore')
  clean_df = clean_df.dropna()

  #Encode explicit column (binary encoding)
  clean_df['explicit'] = clean_df['explicit'].astype(int)

  # Drop track_id, artisits,album_name
  clean_df = clean_df.drop(['track_id', 'artists', 'album_name'], axis=1)

  # Lets look at clustering only track_genre (Pop, Rock,Hip-Hop,Electronic,Indie, blues, Country, Jazz, Latin, Reggae)
  genres_to_keep = ['pop', 'rock', 'hip-hop', 'electronic', 'indie', 'blues ','country', 'jazz', 'latin', 'reggae']
  clean_df = clean_df[clean_df['track_genre'].isin(genres_to_keep)]
  track_genre_mapping = {'pop': 0, 'rock': 1, 'hip-hop': 2, 'electronic': 3, 'indie': 4, 'blues ': 5, 'country': 6, 'jazz': 7, 'latin': 8, 'reggae': 9}
  clean_df['track_genre'] = clean_df['track_genre'].map(track_genre_mapping)

  return clean_df

# Apply the function to the DataFrame
spotify_clean = clean_data(df)

In [ ]:
#Looking at cleaned data frame
spotify_clean.sample(15)

,track_name,popularity,duration_ms,explicit,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,track_genre
32000,Shootout,78,295308,0,0.663,0.578,2,-9.079,1,0.0687,0.31200,0.906000,0.1090,0.553,86.986,4,3
57840,Cate's Brother,0,180589,0,0.735,0.648,1,-4.523,0,0.0334,0.01140,0.000213,0.3090,0.395,97.985,4,4
81645,Feliz Cumpleaños Ferxxo,0,155960,0,0.865,0.573,5,-5.501,1,0.0678,0.08630,0.000000,0.3050,0.563,94.999,4,0
67730,Sucio Y Lento,0,222386,0,0.717,0.758,5,-2.367,0,0.0718,0.37000,0.000303,0.0861,0.960,175.951,4,8
64430,"Baby, It's Cold Outside - Remastered 2004",0,143026,0,0.499,0.304,0,-11.543,1,0.0632,0.81000,0.000000,0.0546,0.521,129.321,4,7
57191,INFERNO,69,133133,0,0.820,0.611,9,-5.020,0,0.1220,0.07660,0.000025,0.0684,0.637,127.883,4,4
81099,"Kanja Poovu Kannala (From ""Viruman"")",74,237558,0,0.720,0.692,8,-6.354,1,0.0894,0.29100,0.000000,0.1770,0.520,110.011,4,0
64364,Let It Snow! Let It Snow! Let It Snow!,0,115240,0,0.484,0.332,1,-10.312,1,0.0370,0.78700,0.000000,0.3730,0.683,134.999,4,7
19500,Grand,70,178973,0,0.583,0.856,6,-5.446,0,0.0445,0.00484,0.000000,0.2240,0.480,168.053,4,6
81805,Moon Rise,66,174056,0,0.756,0.713,3,-6.910,0,0.1220,0.36500,0.000153,0.2280,0.597,92.039,4,0


In [ ]:
#Split into X and y
X = spotify_clean.drop('track_genre', axis=1)
y = spotify_clean['track_genre']

In [ ]:
# Normalize data after the 1st column
ss = StandardScaler()
X_scaled = ss.fit_transform(X[X.columns[1:]])

## Cross Validation to determine number of clusters

## PCA and/or Clustering

If you apply PCA to your data, include code, cumulative variability explained, and scree plot. Explain how you are using PCA in your project for dimensionality reduction or to learn structure in the data.

If you apply clustering, include code. quantitative metrics to evaluate clustering, and how you determined the number of clusters for your data and why. Explain how clustering helped you learn about the structure of data for your project.